In [1]:
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px
import yfinance as yf
import pandas as pd

In [2]:
views = {
          "Raw Data": lambda x, y: x,
          "Percent Change": lambda x, y: x.pct_change(),
          "Rolling Average": lambda x, y: x.rolling(y).mean(),
          "Rolling Variance": lambda x, y: x.rolling(y).var(),
          "Rolling Standard Deviation": lambda x, y: x.rolling(y).var() ** 0.5,
          "Rolling Coefficient of Variation": lambda x, y: (x.rolling(y).var() ** 0.5)
              / (x.rolling(y).mean()),
    }

def format_plotly(fig, calc, data, start, end):
    fig.update_yaxes(title=None)
    fig.update_xaxes(title=None)
    fig_title = f"{calc} of {data} from {start.strftime('%Y/%m/%d')} to {end.strftime('%Y/%m/%d')}"
    fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    title={
      'text': fig_title,
      'y':.95,
      'x':.5,
      'xanchor': 'center',
      'yanchor': 'top'}
    )


cols = ["Open", "Close", "High", "Low", "Volume"]
start_d = datetime.today() - timedelta(days=365)
end_d = datetime.today()

last_tickers = ""

In [3]:
@widgets.interact(calculation=[x for x in views.keys()], data=cols, rolling=list(range(2, 101)),
                  start=widgets.DatePicker(value=start_d), end=widgets.DatePicker(value=end_d), 
                  tickers=widgets.Text()
                  )
def view_chart(calculation = "Raw Data", data="Close", rolling=60, start=start_d, end=end_d, tickers="",):
    global last_tickers, df
    if tickers and tickers[-1] == ",":
        if tickers != last_tickers:
            df = yf.download(tickers, period="max", progress=False)
            last_tickers = tickers

        start_n = datetime(start.year, start.month, start.day)
        end_n = datetime(end.year, end.month, end.day)
        calcs = views[calculation](df[data], rolling)
        selection = calcs.loc[(calcs.index >= start_n) & (calcs.index <= end_n)]
        fig = go.Figure()
        if isinstance(selection, pd.Series):
            fig.add_trace(go.Scatter(x=selection.index, y=selection,mode='lines',name=tickers[:-1]))
        else:
            for item in selection:
                fig.add_trace(go.Scatter(x=selection.index, y=selection[item],mode='lines',name=item.upper()))

        format_plotly(fig,calculation,data,start,end)
        fig.show(autosize=False)


interactive(children=(Dropdown(description='calculation', options=('Raw Data', 'Percent Change', 'Rolling Aver…